In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd drive/MyDrive/chunks

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, Adagrad
from torch.utils.data import DataLoader, Dataset
from torch.autograd.functional import hessian,jacobian
#torch.autograd.set_detect_anomaly(True)
import time


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

class MigrationPredictor(nn.Module):
    def __init__(self, num_feature, num_classes):
        super(MigrationPredictor, self).__init__()
        self.layer_1 = nn.Linear(num_feature, num_classes)
 
    def forward(self,data):
      out = self.layer_1(data)
      return out
 
    def get_probs(self,data):
      #with torch.no_grad():
      return self.forward(data)

#log-likelihood loss with weights
def loss(probs,target,IDs,sample_weights):
    logf = torch.empty(IDs.size()).to(device) 
    for ID in torch.unique(IDs):
        logf[IDs==ID] = torch.logsumexp(probs[IDs==ID],0) 
    return -torch.mean( (probs*target - logf/741) *sample_weights )

/content/drive/MyDrive/chunks


In [ ]:
#accumulate batches to make larger 5x batches and save as parquet to run faster
# this needs to be run only once to create datasets
for batch in range(0,2155,5):
  print(batch)
  new_batch = int(batch/5)
  if batch<2155:
    df = pd.concat([pd.read_stata(f"data_chunk{batch+i}.dta") for i in range(5)])
  else:
    df = pd.concat([pd.read_stata(f"data_chunk{batch+i}.dta") for i in range(1)])
  df['m'] = df['czone_orig']!=df['czone']
  df['wage_hat'] = (df['wage_hat']) /10000
  df['spouse_wage_hat'] = (df['spouse_wage_hat']) /10000
  df['distance_2'] = (df['distance']**2)/10000
  df['distance'] = df['distance']/100
  df.to_parquet(f'../chunks_pickle/df{new_batch}.parquet')



In [ ]:
unique_cz = 741
try:
  model = MigrationPredictor(num_feature=745, num_classes=1) 
  optimizer = Adam(model.parameters(), lr=0.02)
  checkpoint = torch.load('checkpoint.pth')
  epoch = 1 #checkpoint['epoch']
  batch = checkpoint['batch']
  loss_trend = checkpoint['loss_trend']
  total_loss_trend = checkpoint['total_loss_trend']
  mean_loss_old = checkpoint['mean_loss_old']
  beta_m_vector = checkpoint['beta_m_vector']
  beta_f_vector = checkpoint['beta_f_vector']  
  beta_mig_vector = checkpoint['beta_mig_vector']  
  beta_d_vector = checkpoint['beta_d_vector']  
  beta_d2_vector = checkpoint['beta_d2_vector']  
  beta_m_trend = checkpoint['beta_m_trend']
  beta_f_trend = checkpoint['beta_f_trend']  
  beta_mig_trend = checkpoint['beta_mig_trend']  
  beta_d_trend = checkpoint['beta_d_trend']  
  beta_d2_trend = checkpoint['beta_d2_trend']  
  model.load_state_dict(torch.load('checkpoint.pth')['model'])
  model.to(device)
  model.train()
  optimizer.load_state_dict(checkpoint['optimizer'])
  print(f'checkpoint loaded ')
except:
  model = MigrationPredictor(num_feature=745, num_classes=1) 
  model.to(device)

  optimizer = Adam(model.parameters(), lr=0.02)
  mean_loss_old=np.inf
  total_loss_trend=[]
  epoch = 0
  beta_m_trend=[]
  beta_f_trend=[]
  beta_mig_trend=[]
  beta_d_trend=[]  
  beta_d2_trend=[]
while True:
  loss_trend = []
  i=0
  beta_m_vector = []
  beta_f_vector = []
  beta_d_vector = []
  beta_d2_vector= []
  beta_mig_vector = []
  for batch in range(430): #np.random.permutation(430):

      st=time.time()
      i=i+1
      #df=pd.read_stata(f"/content/drive/MyDrive/chunks/data_chunk{batch}.dta")
      #df=pd.read_stata(f"../chunks_pickle/df{batch}.dta")
      #df=pd.read_pickle(f"../chunks_pickle/df{batch}.pkl")
      df = pd.read_parquet(f'../chunks_pickle/df{batch}.parquet') 
      #print(len(df)/741)
      #dimention=N*CZ*M, M=number of variables, N=number of people
      X_train = torch.tensor(df[['wage_hat','spouse_wage_hat']].values.reshape(-1,unique_cz,2)).float().to(device)       
      y_train = torch.tensor(df['migrated'].values.reshape(-1,unique_cz,1)).bool().to(device)
      sample_weights = torch.tensor(df['perwt'].values.reshape((-1,unique_cz,1))).int().to(device)
      czone_fe = torch.tensor(pd.get_dummies(df['czone']).values.reshape(-1,unique_cz,unique_cz)[:,:,0:-1]).bool().to(device)
      m = torch.tensor(df['m'].values.reshape(-1,unique_cz,1)).bool().to(device)
      distance = torch.tensor(df['distance'].values.reshape(-1,unique_cz,1)).int().to(device)
      distance_2 = torch.tensor(df['distance_2'].values.reshape(-1,unique_cz,1)).int().to(device)
      data_input = torch.cat((X_train, m, distance, distance_2, czone_fe), 2).to(device) #TAKE THIS BACK WHEN INCLUDING FEs
      
      IDs = torch.tensor(df['serial'].values.reshape(-1,unique_cz,1).astype(int)).to(device)
      optimizer.zero_grad()
      probs = model.get_probs(data_input).to(device)
      out = loss(probs, y_train, IDs, sample_weights).to(device)
      
      out.backward()
      optimizer.step()

      print(f" beta_m: {list(model.parameters())[0][0][0].item()} ,  beta_f: {list(model.parameters())[0][0][1].item()} ,  beta_d: {list(model.parameters())[0][0][3].item()} , beta_d2: {list(model.parameters())[0][0][4].item()} , loss: {out} , epoch: {epoch}   , batch: {batch}  , time: {time.time()-st} , mean_loss: {mean_loss_old} ")
      loss_trend.append(out.cpu().detach().numpy())
      
      beta_m_vector.append(list(model.parameters())[0][0][0].item())
      beta_f_vector.append(list(model.parameters())[0][0][1].item())
      beta_mig_vector.append(list(model.parameters())[0][0][2].item())
      beta_d_vector.append(list(model.parameters())[0][0][3].item())
      beta_d2_vector.append(list(model.parameters())[0][0][4].item())
      
      #clear cuda cash to avoid CUDA memory runtime error resulting from allocation of memory to variable tensor sizes (didn't need it for when tensor sizes are the same)
      del [data_input,X_train,y_train,m,distance,distance_2,czone_fe,sample_weights]
      torch.cuda.empty_cache() 

  #save checkpoint in case the code is interrupted      
  checkpoint = { 
  'batch': batch,
  'model': model.state_dict(),
  'optimizer': optimizer.state_dict(),
  'beta_m_vector': beta_m_vector,
  'beta_f_vector': beta_f_vector,
  'beta_mig_vector': beta_mig_vector,
  'beta_d_vector': beta_d_vector,
  'beta_d2_vector': beta_d2_vector,
  'beta_m_trend': beta_m_trend,
  'beta_f_trend': beta_f_trend,
  'beta_mig_trend': beta_mig_trend,
  'beta_d_trend': beta_d_trend,
  'beta_d2_trend': beta_d2_trend,
  'loss_trend': loss_trend,
  'beta_mig_trend': beta_mig_trend,
  'beta_d_trend': beta_d_trend,
  'beta_d2_trend': beta_d2_trend,
  'mean_loss_old': mean_loss_old,
  'total_loss_trend': total_loss_trend,
  }
  torch.save(checkpoint, 'checkpoint.pth')
  total_loss_trend.append(out)
  mean_loss_new = np.average(loss_trend)
  epoch = epoch+1
  beta_m_trend.append(beta_m_vector)
  beta_f_trend.append(beta_f_vector)
  beta_mig_trend.append(beta_mig_vector)
  beta_d_trend.append(beta_d_vector)
  beta_d2_trend.append(beta_d2_vector)    
  

  if abs(mean_loss_old-mean_loss_new)<0.001:
      break
  else:
      mean_loss_old=mean_loss_new
  pd.DataFrame({'b_m':beta_m_trend,'b_f':beta_f_trend, 'b_mig':beta_mig_trend, 'b_d':beta_d_trend, 'b_d2':beta_d2_trend}).to_csv('trends_dataframe')

beta_m_final = np.average(beta_m_vector )
beta_f_final = np.average(beta_f_vector)
beta_d_final = np.average(beta_d_vector)
beta_d2_final = np.average(beta_d2_vector)
beta_mig_final = np.average(beta_mig_vector)
print(f"final coefficients: beta_m={beta_m_final},beta_f={beta_f_final},beta_mig={beta_mig_final},beta_d={beta_d_final},beta_d2={beta_d2_final}")

#save trends to convergence
pd.DataFrame({'b_m':beta_m_trend,'b_f':beta_f_trend, 'b_mig':beta_mig_trend, 'b_d':beta_d_trend, 'b_d2':beta_d2_trend, 'epoch':epoch, 'batch':batch}).to_csv('trends_dataframe')


In [ ]:
pd.get_dummies(df['czone']).values.reshape(-1,unique_cz,unique_cz)[:,:,0:-1].shape

(1995, 741, 740)

In [ ]:
  model = MigrationPredictor(num_feature=746, num_classes=1) 
  optimizer = Adam(model.parameters(), lr=0.001)
  checkpoint = torch.load('checkpoint.pth')
  epoch = checkpoint['epoch']
  batch = checkpoint['batch']
  loss_trend = checkpoint['loss_trend']
  total_loss_trend = checkpoint['total_loss_trend']
  mean_loss_old = checkpoint['mean_loss_old']
  beta_m_vector = checkpoint['beta_m_vector']
  beta_f_vector = checkpoint['beta_f_vector']  
  beta_mig_vector = checkpoint['beta_mig_vector']  
  beta_d_vector = checkpoint['beta_d_vector']  
  beta_d2_vector = checkpoint['beta_d2_vector']  
  beta_m_trend = checkpoint['beta_m_trend']
  beta_f_trend = checkpoint['beta_f_trend']  
  beta_mig_trend = checkpoint['beta_mig_trend']  
  beta_d_trend = checkpoint['beta_d_trend']  
  beta_d2_trend = checkpoint['beta_d2_trend']  
  model.load_state_dict(torch.load('checkpoint.pth')['model'])
  model.to(device)
  model.train()
  optimizer.load_state_dict(checkpoint['optimizer'])
  print(f'checkpoint loaded at epoch {epoch}')

KeyError: ignored

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, Adagrad
from torch.utils.data import DataLoader, Dataset
from torch.autograd.functional import hessian,jacobian
#torch.autograd.set_detect_anomaly(True)
import time

%cd drive/MyDrive/chunks

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

class MigrationPredictor(nn.Module):
    def __init__(self, num_feature, num_classes):
        super(MigrationPredictor, self).__init__()
        self.layer_1 = nn.Linear(num_feature, num_classes)
 
    def forward(self,data):
      out = self.layer_1(data)
      return out
 
    def get_probs(self,data):
      #with torch.no_grad():
      return self.forward(data)


#log-likelihood loss with weights
def loss(probs,target,IDs,sample_weights):
    logf = torch.empty(IDs.size()).to(device) 
    for ID in torch.unique(IDs):
        logf[IDs==ID] = torch.logsumexp(probs[IDs==ID],0) 
    return -torch.mean( (probs*target - logf/741) *sample_weights )

model = MigrationPredictor(num_feature=745, num_classes=1) 
optimizer = Adam(model.parameters(), lr=0.01)
checkpoint = torch.load('checkpoint.pth')
#epoch = checkpoint['epoch']
batch = checkpoint['batch']
loss_trend = checkpoint['loss_trend']
total_loss_trend = checkpoint['total_loss_trend']
mean_loss_old = checkpoint['mean_loss_old']
beta_m_vector = checkpoint['beta_m_vector']
beta_f_vector = checkpoint['beta_f_vector']  
beta_mig_vector = checkpoint['beta_mig_vector']  
beta_d_vector = checkpoint['beta_d_vector']  
beta_d2_vector = checkpoint['beta_d2_vector']  
beta_m_trend = checkpoint['beta_m_trend']
beta_f_trend = checkpoint['beta_f_trend']  
beta_mig_trend = checkpoint['beta_mig_trend']  
beta_d_trend = checkpoint['beta_d_trend']  
beta_d2_trend = checkpoint['beta_d2_trend']  
model.load_state_dict(torch.load('checkpoint.pth')['model'])
model.to(device)
model.train()
optimizer.load_state_dict(checkpoint['optimizer'])
#print(f'checkpoint loaded at epoch {epoch}')

def loss0(a):
  total_len = 0
  output = torch.tensor([0]).to(device)
  batches=range(2)
  unique_cz=741
  for batch in range(430):
    print(f"batch={batch}")
    df = pd.read_parquet(f'../chunks_pickle/df{batch}.parquet') 
    total_len = total_len + len(df['serial'])
    X_train0 = torch.tensor(df['wage_hat'].values.reshape(-1,unique_cz,1)).float().to(device)       
    X_train1 = torch.tensor(df['spouse_wage_hat'].values.reshape(-1,unique_cz,1)).float().to(device)       
    y_train = torch.tensor(df['migrated'].values.reshape(-1,unique_cz,1)).bool().to(device)
    sample_weights = torch.tensor(df['perwt'].values.reshape((-1,unique_cz,1))).int().to(device)
    IDs = torch.tensor(df['serial'].values.reshape(-1,unique_cz,1).astype(int)).to(device)

    probs0 = (a[0]*X_train0+a[1]*X_train1)
    logf0 = torch.empty(IDs.size()).to(device) 
    for ID in torch.unique(IDs):
      logf0[IDs==ID] = torch.logsumexp(probs0[IDs==ID],0) 
    output = output -torch.mean( (probs0*y_train - logf0/741)*sample_weights )
    print(output)
    
    #clear cuda cash to avoid CUDA memory runtime error resulting from allocation of memory to variable tensor sizes (didn't need it for when tensor sizes are the same)
    del [X_train0,X_train1,y_train,IDs,probs0,logf0,sample_weights]
    torch.cuda.empty_cache()
    #X_train0 = torch.empty
    #X_train1 = torch.empty
    #y_train = torch.empty
    #sample_weights = torch.empty
    #IDs = torch.empty
    #probs0 = torch.empty
    #logf0 = torch.empty
  return output #/427


torch.sqrt(torch.inverse(hessian(loss0,torch.tensor([list(model.parameters())[0][0][0].item(),list(model.parameters())[0][0][1].item()]))))

/content/drive/MyDrive/chunks
batch=0
tensor([0.3544], device='cuda:0', grad_fn=<SubBackward0>)
batch=1
tensor([0.7709], device='cuda:0', grad_fn=<SubBackward0>)
batch=2


RuntimeError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = MigrationPredictor(num_feature=746, num_classes=1) 
optimizer = Adam(model.parameters(), lr=0.01)
checkpoint = torch.load('checkpoint.pth')
epoch = checkpoint['epoch']
batch = checkpoint['batch']
model.load_state_dict(torch.load('checkpoint.pth')['model'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer'])

output=0
unique_cz=741
df = pd.read_parquet(f'../chunks_pickle/df0.parquet') 
X_train0 = torch.tensor(df['wage_hat'].values.reshape(-1,741,1)).float().to(device)   
X_train1 = torch.tensor(df['spouse_wage_hat'].values.reshape(-1,unique_cz,1)).float().to(device)       
y_train = torch.tensor(df['migrated'].values.reshape(-1,unique_cz,1)).bool().to(device)
sample_weights = torch.tensor(df['perwt'].values.reshape((-1,unique_cz,1))).int().to(device)


IDs = torch.tensor(df['serial'].values.reshape(-1,unique_cz,1).astype(int)).to(device)

optimizer.zero_grad()
probs0 = (list(model.parameters())[0][0][0].item()*X_train0+list(model.parameters())[0][0][1].item()*X_train1)
logf0 = torch.empty(IDs.size()).to(device) 
for ID in torch.unique(IDs):
  logf0[IDs==ID] = torch.logsumexp(probs0[IDs==ID],0) 
output = output -torch.mean( (probs0*y_train - logf0/741)*sample_weights )
output

tensor(0.4400, device='cuda:0')

In [ ]:
#end of main code=========================================================================================
#end of main code=========================================================================================
#end of main code=========================================================================================
#end of main code=========================================================================================

In [ ]:
torch.tensor([list(model.parameters())[0][0][0].item(),list(model.parameters())[0][0][1].item(),list(model.parameters())[1].item()])

In [ ]:
import os
#torch.multiprocessing.set_start_method('forkserver', force=True)
#torch.multiprocessing.set_start_method('spawn', force=True)# good solution !!!!

class MyDataset(Dataset):
    def __init__(self):
        self.data_files = os.listdir()
        #sort(self.data_files)
        #self.y = torch.tensor(self.data_files['migrated'].values.reshape(unique_individuals,unique_cz,1)).float().to(device)
        #self.X = torch.tensor(self.data_files[['wage_hat','spouse_wage_hat']].values.reshape(unique_individuals,unique_cz,2)).float().to(device)   

    def __getindex__(self, idx):
        return pd.read_stata(self.data_files[idx])

    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        df = self.__getindex__(idx)
        df['m'] = df['czone_orig']!=df['czone']
        df['wage_hat'] = (df['wage_hat']) /10000
        df['spouse_wage_hat'] = (df['spouse_wage_hat']) /10000
        #dimention=N*CZ*M, M=number of variables, N=number of people
        X_train = torch.tensor(df[['wage_hat','spouse_wage_hat']].values.reshape(-1,unique_cz,2)).float().to(device)       
        df['migrated'] = df['czone_dest']==df['czone']
        self.y_train = torch.tensor(df['migrated'].values.reshape(-1,unique_cz,1)).bool().to(device)
        self.sample_weights = torch.tensor(df['perwt'].values.reshape((-1,unique_cz,1))).int().to(device)
        #print(time.time()-st)
        czone_fe = torch.tensor(pd.get_dummies(df['czone']).values.reshape(-1,unique_cz,unique_cz)).bool().to(device)
        #print("fe",time.time()-st)
        m = torch.tensor(df['m'].values.reshape(-1,unique_cz,1)).bool().to(device)
        
        self.data_input = torch.cat((X_train, m, czone_fe), 2).to(device) #TAKE THIS BACK WHEN INCLUDING FEs
        
        self.IDs = torch.tensor(df['serial'].values.reshape(-1,unique_cz,1).astype(int)).to(device)
        return self.y_train, self.data_input, self.IDs, self.sample_weights


dset = MyDataset()
loader = DataLoader(dset, num_workers=0, shuffle=True)

In [ ]:
unique_cz = 741

try:
  model = MigrationPredictor(num_feature=744, num_classes=1)#.to(device) #743, num_classes=1) CHNAGE WHEN FEs ARE INCLUDED
  model.to(device)

  optimizer = Adam(model.parameters(), lr=0.2)
  mean_loss_old=np.inf
  mean_loss_trend=[]

except:
  model = torch.load("mymodel")
while True:
    epoch = epoch+1
    loss_trend = []
    i=0
    st=time.time()
    for c, data in enumerate(loader,0):
        print(c)
        
        i=i+1

        y_train, data_input, IDs, sample_weights = data
        
        
        #print("before optimize",time.time()-st)
        optimizer.zero_grad()
        probs = model.get_probs(data_input).to(device)
        out = loss(probs, y_train, IDs, sample_weights) 
        #print("before backward",time.time()-st)
        
        out.backward()
        #print("before step",time.time()-st)

        optimizer.step()
        #print("after optimize",time.time()-st)

        print(f" beta_m: {list(model.parameters())[0][0][0].item()}, beta_f: {list(model.parameters())[0][0][1].item()}, loss: {out}, epoch: {epoch}, batch: {i}th,{batch}, time: {time.time()-st}, mean_loss: {mean_loss_old}")
        #print("after print",time.time()-st)
        loss_trend.append(out)
    mean_loss_new = np.sum(loss_trend)
    torch.save(model,"mymodel")
    if abs(mean_loss_old-mean_loss_new)<0.00001:
        break
    else:
        mean_loss_old=mean_loss_new
    mean_loss_trend.append(mean_loss_new)


In [ ]:
class my_Dataset(Dataset):
    # Characterizes a dataset for PyTorch
    def __init__(self, data_paths, target_paths, transform=None):
        self.data_paths = data_paths
        self.target_paths = target_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, index):
        x = torch.from_numpy(np.load(self.data_paths[index]))
        y = torch.from_numpy(np.load(self.target_paths[index]))
        if self.transform:
            x = self.transform(x)

        return x, y
dataset = my_Dataset(data_paths = os.listdir(),target_paths = os.listdir(), transform=None)
loader = DataLoader(
    dataset,
    batch_size=5,
    shuffle=True,
    num_workers=2
)